In [ ]:
from keras.layers import Conv2D, Input,MaxPool2D, Reshape,Activation,Flatten, Dense, Permute
from keras.models import Model, Sequential
import tensorflow as tf
from keras.layers.advanced_activations import PReLU
import numpy as np
from keras.optimizers import adam

In [ ]:
training_filename = "C:/Users/Seamfix/Documents/Old system/Documents/Seamfix_projects/face-model/Tfrecords/train_PNet_landmark.tfrecord_shuffle.000"
validation_filename = "C:/Users/Seamfix/Documents/Old system/Documents/Seamfix_projects/face-model/Tfrecords/train_PNet_landmark.tfrecord_shuffle.001"
testing_filename = "C:/Users/Seamfix/Documents/Old system/Documents/Seamfix_projects/face-model/Tfrecords/train_PNet_landmark.tfrecord_shuffle.002"

In [ ]:
BATCH_SIZE = 384
buffer_size = 10 * BATCH_SIZE
print("Trial")

In [ ]:
import tensorflow as tf

for example in tf.python_io.tf_record_iterator(training_filename):
    print(tf.train.Example.FromString(example))

In [ ]:
def read_record(example):
    image_features = tf.io.parse_single_example(
       example,
       features={
           'image/encoded': tf.io.FixedLenFeature([], tf.string),#one image  one record
           'image/label': tf.io.FixedLenFeature([], tf.int64),
           'image/roi': tf.io.FixedLenFeature([4], tf.float32),
           'image/landmark': tf.io.FixedLenFeature([10],tf.float32)
       }
    )
    image = tf.io.decode_raw(image_features['image/encoded'], tf.uint8)
    image = tf.reshape(image, [12, 12, 3])
    image = (tf.cast(image, tf.float32)-127.5) / 128
    
    # image = tf.image.per_image_standardization(image)
    label = tf.cast(image_features['image/label'], tf.float32)
    roi = tf.cast(image_features['image/roi'],tf.float32)
    landmark = tf.cast(image_features['image/landmark'],tf.float32)
    return image, (label, roi)

In [ ]:
def get_batch(filenames):
    option_no_order = tf.data.Options()
    option_no_order.experimental_deterministic = False
    
    
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.with_options(option_no_order)
    
    dataset = dataset.interleave(tf.data.TFRecordDataset,cycle_length=1)
    dataset = dataset.map(read_record)
    
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    
    dataset = dataset.prefetch(buffer_size=buffer_size)
    
    return dataset
get_batch(training_filename)

In [ ]:
def get_batch(filenames):
    option_no_order = tf.data.Options()
    option_no_order.experimental_deterministic = False
    
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.with_options(option_no_order)
    
    dataset = dataset.interleave(tf.data.TFRecordDataset,cycle_length=1)
    dataset = dataset.map(read_record)
    print(dataset)
get_batch(training_filename)

In [ ]:
def get_training_datest():
    return get_batch(training_filename)

def get_validation_dataset():
    return get_batch(validation_filename)

def get_testing_dataset():
    return get_batch(testing_filename)

In [ ]:
def create_Kao_Pnet():
    input = Input(shape=[None, None, 3])
    x = Conv2D(10, (3, 3), strides=1, padding='valid', name='conv1')(input)
    x = PReLU(shared_axes=[1,2],name='PReLU1')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = Conv2D(16, (3, 3), strides=1, padding='valid', name='conv2')(x)
    x = PReLU(shared_axes=[1,2],name='PReLU2')(x)
    x = Conv2D(32, (3, 3), strides=1, padding='valid', name='conv3')(x)
    x = PReLU(shared_axes=[1,2],name='PReLU3')(x)
    classifier = Conv2D(2, (1, 1), activation='softmax', name='conv4-1')(x)
    bbox_regress = Conv2D(4, (1, 1), name='conv4-2')(x)
    my_adam = adam(lr = 0.00001)
    model = Model([input], [classifier, bbox_regress])
    model.compile(loss='mse', optimizer=my_adam, metrics=["accuracy"])
    return model

In [ ]:
#initializing model 
modelsss = create_Kao_Pnet()

In [ ]:
# Fit the model
modelsss.fit(get_training_datest(), steps_per_epoch=100, epochs=10, validation_data=get_validation_dataset(), validation_steps=50)

In [ ]:
model.fit(get_training_datest(), ims_roi, roi_score, batch_size=batch_size, epochs=1)

In [ ]:
#initializing model 
modelsss = create_Kao_Pnet()